In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema.runnable import RunnablePassthrough

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler(), ],
    model="gpt-4o-mini"
)

memory = ConversationBufferWindowMemory(
    return_messages=True,
    k=4
)

examples = [
    {
    "question": "Describe Inception in three emojis?",
    "answer": """
🌌🌀🧠
"""
}, 
{
    "question": "Describe Parasite in three emojis?",
    "answer": """
🏠🌧️🍜
"""
},
{
    "question": "Describe The Grand Budapest Hotel in three emojis?",
    "answer": """
🏨🎨🕰️
"""
},
{
    "question": "Describe The Dark Knight in three emojis?",
    "answer": """
🦇🃏🌆
"""
}]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    example_prompt,
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}")
])


def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat

def invoke_chain(input):
    result = chain.invoke({"question":input})
    memory.save_context({"input": input}, {"output": result.content})

invoke_chain("Describe Spirited Away in three emojis?")



🌊👧🐉

In [29]:
invoke_chain("Describe Top Gun in three emojis?")

✈️😎🔥

In [30]:
invoke_chain("Describe The Godfather in three emojis?")

👨‍👧‍👦💼🔫

In [31]:
invoke_chain("Describe Whiplash in three emojis?")

🥁🎶🔥

In [32]:
invoke_chain("Tell me two movies I asked for before.")

You previously asked for descriptions of "Inception" and "Parasite."